In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
# plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

### 데이터 불러오기

In [ ]:
df = pd.read_csv('../Pizza Sales Dataset.csv')
df

In [ ]:
df.columns

### 컬럼명 한국어로 바꾸기

In [ ]:
df.rename(columns={
    'pizza_id': '피자 ID',
    'order_id': '주문 ID',
    'pizza_name_id': '피자 이름 ID',
    'quantity': '주문 수량',
    'order_date': '주문 날짜',
    'order_day': '주문 요일',
    'order_time': '주문 시간',
    'unit_price': '단가',
    'total_price': '총액',
    'pizza_size': '피자 사이즈',
    'pizza_category': '피자 종류',
    'pizza_ingredients': '피자 재료 목록',
    'pizza_name': '피자 이름'
}, inplace=True)

df

In [ ]:
df['피자 이름'].value_counts()

In [ ]:
# 결측치 확인 : 없음
df.isna().sum()

In [ ]:
df.info()

### 분석 시작

### EDA

---------------------------------------------------------------------------------

### 수연님 Part

 ### 🍕주문 ID가 같은 사용자 중(한번에 여러판 주문) 같이 주문하는 피자의 조합 중 BEST 5를 뽑아보자.

In [ ]:
# 미리 알아두어야 할 것.. 한 판 씩 시킨 주문이 압도적으로 많음

# 두 종류 이상의 피자가 같이 주문 된 경우만 필터링
# multi_combi : 한번에 두 판 이상 시킨 주문 data
df_subset = df[['주문 ID', '피자 이름']]
order_combi = df_subset.groupby('주문 ID')['피자 이름'].apply(list)
order_combi = order_combi.apply(lambda x: tuple(sorted(x)))
multi_combi = order_combi[order_combi.apply(lambda x: len(x) > 1)]

from collections import Counter
multi_combi_counter = Counter(multi_combi)
top3_multi_combi = multi_combi_counter.most_common(5)

top3_multi_combi_df = pd.DataFrame(top3_multi_combi, columns = ['피자조합', "빈도수"])
display(top3_multi_combi_df)

top3_multi_combi_df.to_csv('한번에 함께 주문된 피자조합 top5', index = False, encoding="utf-8-sig")

In [ ]:
# 튜플을 보기 좋게 문자열로 바꾸기 (예: 피자1\n피자2)
top3_multi_combi_df["피자조합_이름"] = top3_multi_combi_df["피자조합"].apply(lambda x: "\n".join(x))

# seaborn 불러오기
import seaborn as sns

# seaborn을 사용한 가로 막대 그래프 시각화
plt.figure(figsize=(8, 5))
sns.barplot(
    data=top3_multi_combi_df,
    y="피자조합_이름",
    x="빈도수",
    palette="autumn"
)

plt.xlabel("주문 빈도수")
plt.ylabel("피자 조합")
plt.title("한 번에 함께 주문된 피자 조합 TOP 5 (Seaborn)")
plt.tight_layout()
plt.show()

---------------------------------------------------------------------------------

### **🕹️가격대별 고객 취향 세분화 (클러스터링 유사 접근)**

- **목표**: 고객들은 어떤 가격대 + 어떤 종류의 피자를 즐기는가?
- **시각화**: `unit_price`, `category`, `size` 기준 버블 차트
- **활용**: 타깃 고객별 맞춤 프로모션 (ex. 저가+소형 선호 고객층 vs 고가+XL 고객층)

In [ ]:
# 가격 구간화 

def 가격구간(가격):
    if 가격 <= 14.99:
        return '저가 (≤ $14.99)'
    elif 15.00 <= 가격 <= 19.99:
        return '중가 ($15.00 ~ $19.99)'
    else:
        return '고가 (≥ $20.00)'

# 단가 기준으로 구간 분류
df['가격 구간'] = df['단가'].apply(가격구간)

# 가격 구간 + 피자 종류별 수량 합계
grouped = df.groupby(['가격 구간', '피자 종류'])['주문 수량'].sum().reset_index()

# 각 가격 구간에서 총 수량 기준으로 비율 계산
grouped['비율'] = grouped.groupby('가격 구간')['주문 수량'].transform(lambda x: round(x / x.sum(), 2))
grouped

grouped.to_csv('가격대별 피자 종류 분석', index = False, encoding = 'utf-8-sig')

In [ ]:
# 누적 barplot으로 그려보기

pivot_df = grouped.pivot(index='가격 구간', columns='피자 종류', values='비율')

pivot_df.plot(kind='bar', stacked=True, figsize=(8, 7), colormap='tab20')
plt.title('가격 구간별 피자 종류 선호 비율')
plt.ylabel('비율')
plt.xlabel('가격 구간')
plt.legend(title='피자 종류', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
# 파이그래프로 그려보기
import matplotlib.pyplot as plt

# 각 가격 구간에 대해 pie chart 반복 생성
구간_리스트 = grouped['가격 구간'].unique()

for 구간 in 구간_리스트:
    # 현재 구간의 데이터 필터링
    temp = grouped[grouped['가격 구간'] == 구간]

    labels = temp['피자 종류']
    sizes = temp['비율']  # 이미 0~1 사이 비율로 되어 있음

    # 차트 생성
    plt.figure(figsize=(6, 6))
    plt.pie(
        sizes, 
        labels=labels, 
        autopct='%1.0f%%', 
        startangle=90, 
        counterclock=False, 
        wedgeprops={'edgecolor': 'white'}
    )
    plt.title(f"{구간} 피자 종류 구성비")
    plt.tight_layout()
    plt.show()


**🔍 인사이트 정리 (보고서/발표용)**

✅ 1. 저가 구간 (≤ $14.99)
Classic 피자 비중 압도적 (~50%)

그 외에는 Supreme, Chicken이 균형 있게 분포

📝 인사이트:

저가 라인은 "전통적인 입맛" 중심. Classic 피자를 중심으로 단일 인기 메뉴 전략이 가능. 원가 절감형 베스트셀러 구성에 적합.

✅ 2. 중가 구간 ($15.00 ~ $19.99)
4가지 종류가 골고루 분포
(Chicken, Classic, Supreme, Veggie 균형 잡힘)

📝 인사이트:

중가 라인은 취향 다양성이 극대화된 구간. 모든 카테고리 피자를 고르게 즐김 → 콤보 메뉴, 혼합 패키지, 커스터마이징 옵션 강화에 적합.

✅ 3. 고가 구간 (≥ $20.00)
Chicken 비율 상승, Classic 감소

Veggie/Supreme도 고르게 유지

📝 인사이트:

고가 피자는 육류 중심의 프리미엄 취향이 뚜렷함. Chicken 피자에 대한 선호가 확연히 드러남. 고단백/고급 토핑 기반 프리미엄 라인 확장 가능.

---------------------------------------------------------------------------------

### **🧄토핑 종류 다양성과 단가 관계 분석 수연님**

- 목표: 토핑이 많을수록 가격이 비쌀까?
- 전처리: Toppings 개수 계산 → Unit Price와 관계 산점도
- 활용: 토핑 추가 가격 정책 검토

In [ ]:
# 토핑 문자열을 , 기준으로 분리한 후 개수 계산
df['토핑 수'] = df['피자 재료 목록'].apply(lambda x: len(x.split(',')))

In [ ]:
# 단가와 토핑수를 산점도로 나타내기

plt.figure(figsize=(8, 5))
plt.scatter(df['토핑 수'], df['단가'], alpha=0.6)
plt.title('토핑 수 vs 단가 관계')
plt.xlabel('토핑 수')
plt.ylabel('단가 ($)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
corr = df[['토핑 수', '단가']].corr().iloc[0, 1]
print(f"토핑 수와 단가 간 상관계수: {corr:.2f}")

**📌 분석 요약**

X축: 피자당 토핑 수 (2~8개)

Y축: 해당 피자의 단가 ($)

상관계수: 0.35 → 약한~중간 정도의 양의 상관관계

**🔍 데이터 인사이트**

✅ 1. 토핑 수가 많을수록 가격이 오르는 경향
대체로 토핑 수 5~7개일 때 단가가 높은 피자들이 많음

**최고가 피자($35 이상)**도 토핑 6개인 구간에서 등장

✅ 2. 예외 케이스 존재
토핑 수 5~6개지만 단가가 낮은 피자 존재

토핑 수 2~3개임에도 단가가 높은 피자도 일부 존재 → 브랜드 전략 또는 고급 재료 가능성

💬 인사이트 요약 문장 (보고서용)
“토핑 수와 단가는 중간 정도의 양의 상관관계를 보이며, 일반적으로 토핑이 많을수록 가격이 높아지는 경향이 있습니다. 다만 일부 토핑 수가 적어도 가격이 높은 제품이 존재하며, 이는 브랜드 가치나 프리미엄 원재료 사용 등의 요인이 반영된 것으로 해석됩니다.”

---------------------------------------------------------------------------------

### 효준님 Part

In [ ]:
# 중복 제거된 피자 이름과 재료 목록 추출
pizza_ingredients_count = df[['피자 이름', '피자 재료 목록']].drop_duplicates()

# 재료 수 계산
pizza_ingredients_count['재료 수'] = pizza_ingredients_count['피자 재료 목록'].apply(lambda x: len(x.split(',')))

# 재료 수 기준 내림차순 정렬
pizza_ingredients_count_sorted = pizza_ingredients_count.sort_values(by='재료 수', ascending=False)

# 전체 출력
print(pizza_ingredients_count_sorted.to_string(index=False))

In [ ]:
# ✅ 재료 수 계산 (쉼표 기준 split해서 개수 세기)
df['재료 수'] = df['피자 재료 목록'].apply(lambda x: len(x.split(',')))

# ✅ 재료 수별 총 주문 수량 계산
ingredient_group_total = df.groupby('재료 수')['주문 수량'].sum().reset_index()

# ✅ 시각화
plt.figure(figsize=(8, 5))
sns.lineplot(data=ingredient_group_total, x='재료 수', y='주문 수량', marker='o')
plt.title('🧂 토핑 재료 수 vs 총 주문 수량')
plt.xlabel('토핑 재료 수')
plt.ylabel('총 주문 수량')
plt.grid(True)
plt.tight_layout()
plt.show()

🧂 인사이트: 토핑 재료 수와 주문량 관계 분석
📈 관찰 요약
재료 수가 4~6개인 피자가 가장 많은 주문량을 기록함.

재료 수가 2~3개인 단순한 피자는 주문량이 적은 편.

7개 이상 재료를 가진 피자는 개수는 적지만 상대적으로 높은 주문량을 기록하는 경우도 존재함.

🔍 분석 포인트
재료 수	주문 수량 특성	해석
1~3개	낮은 주문 수량	지나치게 단순한 메뉴는 선호도가 낮음
4~6개	주문량 최고 구간	맛의 다양성과 적절한 가격대의 균형 가능성
7개 이상	다소 줄어드는 추세 or 소수 인기	프리미엄 포지션의 가능성, 다만 너무 복잡한 재료는 호불호 존재

💡 마케팅 & 제품 전략 제안
✅ 메인 메뉴군은 4~6개 재료를 사용하는 피자로 구성

→ 예: 베스트셀러, 추천메뉴, 세트 구성용

✅ 단순 피자는 1인 런치용 소형 사이즈로 리포지셔닝

→ 예: 저가 간편식, 배달 전용 메뉴

✅ 7개 이상 재료의 피자는 프리미엄 라인업으로 별도 강조

→ 예: “쉐프 스페셜”, “고급 토핑 한정 메뉴” 등

✨ 활용 방안
신메뉴 개발 시 재료 개수 타겟 범위는 4~6개로 설정

재료 수에 따른 제작 원가, 소비자 인식, 프로모션 전략 연계 가능

향후 재료 조합별 만족도 조사 또는 리뷰 기반 정성 분석도 고려 가능

In [ ]:
# 피자 이름별로 고정 단가와 총 주문 수량 추출
pizza_price_quantity = df.groupby('피자 이름').agg({
    '단가': 'first',         # 고정 단가
    '주문 수량': 'sum'       # 총 주문 수량
}).reset_index()

# 총 주문 수 기준 내림차순 정렬
pizza_price_quantity_sorted = pizza_price_quantity.sort_values(by='주문 수량', ascending=False)

# 표 출력
print(pizza_price_quantity_sorted.to_string(index=False))

In [ ]:
# 단가 구간 분류 함수
def 가격_구간(가격):
    if 가격 <= 14.99:
        return '저가 (≤ $14.99)'
    elif 15.00 <= 가격 <= 19.99:
        return '중가 ($15.00 ~ $19.99)'
    else:
        return '고가 (≥ $20.00)'

# 단가 기준으로 구간 분류 컬럼 추가
df['가격 구간'] = df['단가'].apply(가격_구간)

# 각 가격 구간별 총 주문 수량 계산
price_group_summary = df.groupby('가격 구간')['주문 수량'].sum().reset_index()

# 보기 좋게 가격 구간 정렬
price_group_summary = price_group_summary.sort_values(by='주문 수량', ascending=False)

# 결과 확인
print(price_group_summary.to_string(index=False))


In [ ]:
# 3. 구간별 **총 주문 수량** 집계 (막대차트용으로도 동일하게 사용)
price_group_summary = df.groupby('가격 구간')['주문 수량'].sum().reset_index()

# --------------- 그래프 1: 파이차트 ----------------
plt.figure(figsize=(5, 5))
plt.pie(price_group_summary['주문 수량'], labels=price_group_summary['가격 구간'],
        autopct='%1.1f%%', startangle=90)
plt.title('가격 구간별 총 주문 수량 비중 (파이차트)')
plt.axis('equal')
plt.tight_layout()
plt.show()

# --------------- 그래프 2: 막대차트 ----------------
plt.figure(figsize=(7, 5))
sns.barplot(data=price_group_summary, x='가격 구간', y='주문 수량')
plt.title('가격 구간별 총 주문 수량 (막대차트)')
plt.xlabel('가격 구간')
plt.ylabel('총 주문 수량')
plt.tight_layout()
plt.show()

# --------------- 그래프 3: 산점도 ----------------
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x='단가', y='주문 수량', hue='가격 구간', palette='Set2', alpha=0.6)
plt.title('단가 vs 주문 수량 (산점도)')
plt.xlabel('피자 단가 ($)')
plt.ylabel('주문 수량')
plt.legend(title='가격 구간')
plt.grid(True)
plt.tight_layout()
plt.show()


🧠 1. 가격 구간별 총 주문 수량 비중 (파이차트)
📊 그래프 요약:
저가 (≤ $14.99) 피자가 전체 판매량의 가장 큰 비중을 차지

고가 (≥ $20.00)는 상대적으로 소수

🔍 인사이트:
저가 피자가 전체 판매량의 주력 상품입니다.
→ 단품/점심용/1인용 등으로 자주 팔리는 구조일 수 있음

고가 피자 수요도 존재하지만 소수
→ 마진율은 높을 수 있으나 수요는 제한적 → 프리미엄은 한정 판매 전략 적합

🧠 2. 가격 구간별 총 주문 수량 (막대차트)
📊 그래프 요약:
세 구간 중 저가 피자 > 중가 피자 > 고가 피자 순으로 많이 팔림

고가 피자는 판매량이 확실히 떨어짐

🔍 인사이트:
전체 수요 기반 전략에서는 중저가 라인이 핵심

중가 피자도 꽤 견고한 수요가 있음 → 가족 단위/공동 소비

고가 피자는 적은 수요지만 고정 고객층 가능성
→ 브랜드 피자, 특별한 날, 고급 재료 위주 마케팅이 어울림

🧠 3. 단가 vs 주문 수량 (산점도)
📊 그래프 요약:
단가가 낮을수록 전반적으로 주문 수량이 높은 경향

고가 피자(≥$20)는 대부분 수량이 낮음 (희소하게 몇 개만 조금 팔림)

🔍 인사이트:
단가가 높아질수록 수요가 급격히 낮아지는 경향이 뚜렷
→ 가격 민감도가 높은 시장

단가를 약간 낮춰도 주문 수 증가가 기대될 수 있음
→ $20 근처 제품은 가격 전략 조정 필요성 있음

In [ ]:
# 피자 사이즈별 총 주문 수량 계산
size_quantity = df.groupby('피자 사이즈')['주문 수량'].sum().reset_index()

# 주문 수량 기준 내림차순 정렬
size_quantity = size_quantity.sort_values(by='주문 수량', ascending=False)

# 표 출력
print(size_quantity.to_string(index=False))


In [ ]:
# 1. 피자 사이즈별 총 주문 수량 계산 및 정렬
size_quantity = df.groupby('피자 사이즈')['주문 수량'].sum().reset_index()
size_quantity = size_quantity.sort_values(by='주문 수량', ascending=False)

# 2. 파이차트: 사이즈별 비중
plt.figure(figsize=(6, 6))
plt.pie(
    size_quantity['주문 수량'],
    labels=size_quantity['피자 사이즈'],
    autopct='%1.1f%%',
    startangle=90
)
plt.title('📐 피자 사이즈별 주문 비중 (파이차트)')
plt.axis('equal')
plt.tight_layout()
plt.show()

# 3. 막대차트: 사이즈별 주문 수량
plt.figure(figsize=(8, 5))
sns.barplot(data=size_quantity, x='피자 사이즈', y='주문 수량', palette='Set3')
plt.title('📐 피자 사이즈별 주문 수량 (막대차트)')
plt.xlabel('피자 사이즈')
plt.ylabel('총 주문 수량')
plt.tight_layout()
plt.show()

가격 구간	총 주문 수량	비중
중가 ($15~$19.99)	17,986	37.0%
저가 (≤ $14.99)	16,318	33.6%
고가 (≥ $20)	15,270	29.4%

인사이트:
가격대가 고르게 분포됨 → 고객층 다양

중가 라인은 매출 안정화, 저가는 고객 유입, 고가는 마진 확보

세 가지 라인을 전략적으로 분화하여 포지셔닝 필요

In [ ]:
# 피자 종류별 주문 수량 집계
category_quantity = df.groupby('피자 종류')['주문 수량'].sum().reset_index()

# 주문 수량 기준 내림차순 정렬
category_quantity = category_quantity.sort_values(by='주문 수량', ascending=False)

# 표 출력
print(category_quantity.to_string(index=False))

In [ ]:
# 1. 피자 종류별 주문 수량 집계
category_quantity = df.groupby('피자 종류')['주문 수량'].sum().reset_index()
category_quantity = category_quantity.sort_values(by='주문 수량', ascending=False)

# 2. 막대차트
plt.figure(figsize=(8, 5))
sns.barplot(data=category_quantity, x='피자 종류', y='주문 수량', palette='Set2')
plt.title('📊 피자 종류별 주문 수량 (막대차트)')
plt.xlabel('피자 종류')
plt.ylabel('총 주문 수량')
plt.tight_layout()
plt.show()

# 3. 파이차트
plt.figure(figsize=(6, 6))
plt.pie(
    category_quantity['주문 수량'],
    labels=category_quantity['피자 종류'],
    autopct='%1.1f%%',
    startangle=90
)
plt.title('📊 피자 종류별 주문 비중 (파이차트)')
plt.axis('equal')
plt.tight_layout()
plt.show()

사이즈별 판매 비중
사이즈	총 주문 수량	비중 추정
L	18,956	39%
M	15,635	32%
S	14,403	29%

인사이트:
L 사이즈가 압도적 1위 → 가족, 친구와 나눠먹는 구성 선호

M, S도 비중 큼 → 1~2인 소비층도 충분히 존재

대형 프로모션은 L 사이즈 중심, 소형은 점심/1인 가구 타깃

In [ ]:
## 인기 피자 TOP 10 추출


# 피자 이름별 총 주문 수량 계산
pizza_top10 = df.groupby('피자 이름')['주문 수량'].sum().reset_index()

# 주문 수량 기준 내림차순 정렬 후 상위 10개 추출
pizza_top10 = pizza_top10.sort_values(by='주문 수량', ascending=False).head(10)

# TOP 10 피자 정보 추출 (피자 이름 기준으로 원본과 merge)
pizza_info = df.drop_duplicates(subset='피자 이름')[['피자 이름', '피자 종류', '피자 사이즈', '단가', '피자 재료 목록']]
pizza_top10_full = pd.merge(pizza_top10, pizza_info, on='피자 이름', how='left')

# 재료 수 컬럼 추가
pizza_top10_full['재료 수'] = pizza_top10_full['피자 재료 목록'].apply(lambda x: len(x.split(',')))

# 정리된 표 출력
print(pizza_top10_full.to_string(index=False))

In [ ]:
##단가/재료/사이즈/카테고리별 피자 효율 분석

# 피자 이름별 평균 주문 수량 계산
pizza_summary = df.groupby('피자 이름')['주문 수량'].mean().reset_index()

# 고유 피자 정보 병합 (중복 제거)
pizza_info_full = df.drop_duplicates(subset='피자 이름')[['피자 이름', '단가', '피자 사이즈', '피자 종류', '피자 재료 목록']]

# 재료 수 컬럼 추가
pizza_info_full['재료 수'] = pizza_info_full['피자 재료 목록'].apply(lambda x: len(x.split(',')))

# 병합: 피자별 요약 테이블
pizza_efficiency = pd.merge(pizza_summary, pizza_info_full, on='피자 이름', how='left')

# 주문 수량 기준 정렬
pizza_efficiency = pizza_efficiency.sort_values(by='주문 수량', ascending=False)

# 표 출력
print(pizza_efficiency.to_string(index=False))

In [ ]:
## 어떤 토핑이 제일 많이 들어갔을까

from collections import Counter

# 모든 피자의 재료 목록을 하나의 리스트로 펼치기
all_ingredients = df['피자 재료 목록'].apply(lambda x: [i.strip() for i in x.split(',')])
flat_list = [ingredient for sublist in all_ingredients for ingredient in sublist]

# 재료별 등장 횟수 카운트
ingredient_counts = Counter(flat_list)

# DataFrame으로 정리
ingredient_df = pd.DataFrame(ingredient_counts.items(), columns=['재료명', '등장 횟수'])

# 등장 횟수 기준 내림차순 정렬
ingredient_df = ingredient_df.sort_values(by='등장 횟수', ascending=False).reset_index(drop=True)

# 표 출력
print(ingredient_df.to_string(index=False))


토핑 재료 등장 순위 TOP 10
재료명	등장 횟수	포지션
Garlic	27,422	거의 모든 피자 기본
Tomatoes	26,601	소스/재료 전방위 활용
Red Onions	19,547	중간 토핑
Red Peppers	16,284	상단 토핑
Mozzarella Cheese	10,333	핵심 치즈
Pepperoni	10,300	고기 토핑 대표

인사이트:
마늘, 토마토, 양파는 핵심 베이스 → 공급 안정성과 원가 관리 필요

페퍼로니, 치즈는 고급 피자 핵심 토핑

재료 등장 순서를 바탕으로 공통 베이스 피자 조합 (예: 마늘+토마토+치즈) 개발 가능

 종합 마케팅/제품 전략 제안
전략 영역	제안 내용
제품 개발	- 인기 재료 기반 베이스 조합 최적화
- Veggie 시리즈 라인 확장 (건강식/비건 타깃)
- M사이즈 중심 점심 전용 메뉴 개발
메뉴 포지션	- 중가 Classic & Veggie 중심
- 고가 프리미엄은 한정판/특별 테마로 고정비 방지
마케팅	- "건강한 채식 피자도 맛있다!" 테마
- "한 판이면 충분해" L사이즈 강조
- "우리의 베이스는 다르다" 핵심 재료 강조형 캠페인
공급전략	- Garlic, Tomatoes, Onion은 계약재배 or 대량구매로 원가 절감
- 페퍼로니/치즈는 프리미엄 라인 전용으로 분리 관리

In [ ]:
# 1. 피자 이름별 단가 + 총 주문 수량
#pizza_price_quantity_sorted.to_csv('피자_별_단가_및_총_주문수량.csv', index=False, encoding='utf-8-sig')

# 2. 가격 구간별 총 주문 수량
#price_group_summary.to_csv('가격_구간별_총_주문수량.csv', index=False, encoding='utf-8-sig')

# 3. 토핑 재료 수 vs 총 주문 수량
#ingredient_group_total.to_csv('재료수_별_총_주문수량.csv', index=False, encoding='utf-8-sig')

# 4. 피자 이름별 재료 수
#ingredient_count_sorted.to_csv('피자_별_재료_수.csv', index=False, encoding='utf-8-sig')

# 5. 토핑 종류별 등장 횟수
#topping_count_df.to_csv('토핑_종류별_등장횟수.csv', index=False, encoding='utf-8-sig')


In [ ]:
# 피자 이름별로 토핑 재료 수 계산
ingredient_count = df.groupby('피자 이름')['피자 재료 목록'].first().apply(lambda x: len(x.split(','))).reset_index()
ingredient_count.columns = ['피자 이름', '재료 수']

# 재료 수 기준으로 정렬
ingredient_count_sorted = ingredient_count.sort_values(by='재료 수', ascending=False)


In [ ]:
# 모든 피자의 재료 리스트 합치고, 재료별 등장 횟수 세기
from collections import Counter
topping_series = df['피자 재료 목록'].str.split(',').explode().str.strip()
topping_counts = Counter(topping_series)
topping_count_df = pd.DataFrame(topping_counts.items(), columns=['토핑', '등장 횟수']).sort_values(by='등장 횟수', ascending=False)

---------------------------------------------------------------------------------

### 설연님 Part 
: 컬럼명 달라서 다른 파일로 봐야함

---------------------------------------------------------------------------------

### 현석님 Part

---------------------------------------------------------------------------------

### 혜원님 Part